# TAXI_v2

In [1]:
import gym
import numpy as np

In [2]:
#create a single game instance
env = gym.make("Taxi-v2")

n_states = env.observation_space.n
n_actions = env.action_space.n

In [70]:
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.choice(range(n_actions), n_states)

In [71]:
def sample_reward(env, policy, t_max=200):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    state = env.reset()
    total_reward = 0
    counter = 0
    done = False
    
    while(not done and counter <= t_max):
        state, reward, done, _ = env.step(policy[state])
        total_reward += reward
        counter += 1
        
    return total_reward

In [72]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    return float(np.mean([sample_reward(env, policy) for _ in range(n_times)]))

## Part I Random policies

# -------   L O N G T I M E    -------

In [73]:
# policies = [get_random_policy() for _ in range(10**3)]
# scores = [evaluate(policy) for policy in policies]

# best_policy = None
# best_score = -float('inf')

# best_score = np.max(scores)
# best_policy = policies[np.argmax(scores)]

# print("New best score:", score)

### -607.97 (LUL)

# Part II Genetic algorithm 

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [74]:
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    policy = np.zeros_like(policy1)
    mask = np.random.choice([0, 1], size=len(policy), p=[1-p, p]) == 1
    policy[mask] = policy1[mask]
    policy[~mask] = policy2[~mask]
    return policy  # P(policy_1_) = p !

In [75]:
def mutation(policy, p=0.8):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(get_random_policy(), policy, p)    

In [76]:
def gen_algo(n_epochs, pool_size, n_crossovers, n_mutations, seed, prev_pool=None, prev_scores=None):
    np.random.seed(seed)
    if not prev_pool:
#         policy = np.hstack([np.random.choice(range(4), size=n_states-4), np.array([4,4,5,5])])
#         np.random.shuffle(policy)
#         print(policy.shape)
        pool = [get_random_policy() for _ in range(pool_size)]
        pool_scores = [evaluate(policy) for policy in pool]
    else:
        pool = prev_pool
        prev_scores = prev_pool
    for epoch in range(n_epochs):
        print("Epoch %s:"%epoch)

        to_cross = np.random.choice(range(pool_size), n_crossovers * 2, replace=False)
        crossovered = [crossover(pool[to_cross[ix]], pool[to_cross[-ix-1]]) for ix in range(n_crossovers)]

        mutated = [mutation(pool[ix]) for ix in np.random.choice(pool_size, n_crossovers, replace=False)]

        assert type(crossovered) == type(mutated) == list

        #add new policies to the pool
        mutated += crossovered
        pool += mutated
        pool_scores = [evaluate(policy) for policy in pool]

        #select pool_size best policies
        selected_indices = np.argsort(pool_scores)[-pool_size:]
        pool = [pool[i] for i in selected_indices]
        pool_scores = [pool_scores[i] for i in selected_indices]

        #print the best policy so far (last in ascending score order)
        print("best score:", pool_scores[-1])
    return pool, pool_scores

# ---- D O L G O ----

In [ ]:
pool, pool_scores = gen_algo(150, 50, 20, 40, 911)

Epoch 0:
('best score:', -918.29)
Epoch 1:
('best score:', -988.85)
Epoch 2:
('best score:', -898.04)
Epoch 3:
('best score:', -881.66)
Epoch 4:
('best score:', -792.38)
Epoch 5:
('best score:', -898.85)
Epoch 6:
('best score:', -791.75)
Epoch 7:
('best score:', -846.2)
Epoch 8:
('best score:', -863.93)
Epoch 9:
('best score:', -828.83)
Epoch 10:
('best score:', -827.48)
Epoch 11:
('best score:', -863.48)
Epoch 12:
('best score:', -775.46)
Epoch 13:
('best score:', -828.65)
Epoch 14:
('best score:', -774.56)
Epoch 15:
('best score:', -720.83)
Epoch 16:
('best score:', -649.01)
Epoch 17:
('best score:', -665.66)
Epoch 18:
('best score:', -666.83)
Epoch 19:
('best score:', -737.93)
Epoch 20:
('best score:', -756.74)
Epoch 21:
('best score:', -665.66)
Epoch 22:
('best score:', -630.29)
Epoch 23:
('best score:', -523.1)
Epoch 24:
('best score:', -613.19)
Epoch 25:
('best score:', -577.19)
Epoch 26:
('best score:', -703.19)
Epoch 27:
('best score:', -631.19)
Epoch 28:
('best score:', -612.8

In [ ]:
pool, pool_scores = gen_algo(300, 50, 20, 40, 911, pool, pool_scores)

Epoch 0:
('best score:', -200.0)
Epoch 1:
('best score:', -200.0)
Epoch 2:
('best score:', -200.0)
Epoch 3:
('best score:', -200.0)
Epoch 4:
('best score:', -200.0)
Epoch 5:
('best score:', -200.0)
Epoch 6:
('best score:', -200.0)
Epoch 7:
('best score:', -200.0)
Epoch 8:
('best score:', -200.0)
Epoch 9:
('best score:', -200.0)
Epoch 10:
('best score:', -200.0)
Epoch 11:
('best score:', -200.0)
Epoch 12:
